In [1]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [2]:
narrative_path = '../data_to_model/keras_data/model_df_full_narrative.csv'
train_path = '../data_to_model/Train and Test Sets/train_df_count.csv'

In [3]:
narrative_df = pd.read_csv(narrative_path)
narrative_df.drop(columns= ['primary_mechanism'], inplace = True )

In [4]:
train_df = pd.read_csv(train_path).merge(narrative_df,how='left', on=['cpsc_case_number'] )
#train_df

In [5]:
text_np = np.array(train_df['narrative'])
train_labels_np = np.array(train_df['primary_mechanism'])
#train_labels_np_bin = np.where(train_labels_np == 'Multi-sport', 'Multi-sport', 'Other')
print(train_labels_np[0:30])
#print(train_labels_np_bin[0:30])

['cardio/sync' 'Hit with various obj' 'cardio/sync' 'cardio/sync'
 'cardio/sync' 'cardio/sync' 'cardio/sync' 'cardio/sync' 'cardio/sync'
 'Hit with various obj' 'cardio/sync' 'Undetermined/unknown' 'cardio/sync'
 'Undetermined/unknown' 'Undetermined/unknown' 'Undetermined/unknown'
 'Hit with various obj' 'cardio/sync' 'cardio/sync' 'cardio/sync'
 'Undetermined/unknown' 'Multi-sport' 'Undetermined/unknown' 'cardio/sync'
 'cardio/sync' 'cardio/sync' 'Multi-sport' 'Multi-sport'
 'Undetermined/unknown' 'cardio/sync']


In [6]:
#l_list = []
#for txt in text_np:
#    l_list.append( len(txt) )
    
#l_list[0:10]

#max(l_list)

In [7]:
maxlen = 207
training_samples = 1491
validation_samples = 500
max_words = 100000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_np)
sequences = tokenizer.texts_to_sequences(text_np)


In [8]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', train_labels_np.shape)


Found 1690 unique tokens.
Shape of data tensor: (1696, 207)
Shape of label tensor: (1696,)


In [9]:
#indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
#data = data[indices]
#train_labels_np = train_labels_np[indices]
#x_train = data[:training_samples]
#y_train = train_labels_np[:training_samples]
#x_val = data[training_samples: training_samples + validation_samples]
#y_val = train_labels_np[training_samples: training_samples + validation_samples]

In [10]:
label_encoder = LabelEncoder()

y_train_integer_encoded = label_encoder.fit_transform(train_labels_np)
#train_labels_npy_val_integer_encoded = label_encoder.fit_transform(y_val)

#y_train_integer_encoded
#y_train_integer_encodedtrain_labels_np
y_train_one_hot = to_categorical(y_train_integer_encoded, num_classes=7)
#y_val_one_hot = to_categorical(y_val_integer_encoded, num_classes=7)

In [11]:
y_train_integer_encoded

array([6, 2, 6, ..., 5, 4, 4])

In [12]:
y_train_one_hot

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [13]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(7, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 207, 8)            800000    
_________________________________________________________________
flatten (Flatten)            (None, 1656)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 11599     
Total params: 811,599
Trainable params: 811,599
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(data, y_train_one_hot,
epochs=80,
batch_size=32,
validation_split=.1)

Epoch 1/80
48/48 [==============================] - 1s 13ms/step - loss: 0.3915 - accuracy: 0.5262 - val_loss: 0.4985 - val_accuracy: 0.0000e+00
Epoch 2/80
48/48 [==============================] - 0s 9ms/step - loss: 0.3074 - accuracy: 0.5374 - val_loss: 0.4847 - val_accuracy: 0.0000e+00
Epoch 3/80
48/48 [==============================] - 0s 9ms/step - loss: 0.2934 - accuracy: 0.5387 - val_loss: 0.4667 - val_accuracy: 0.0000e+00
Epoch 4/80
48/48 [==============================] - 0s 9ms/step - loss: 0.2743 - accuracy: 0.5760 - val_loss: 0.4690 - val_accuracy: 0.0000e+00
Epoch 5/80
48/48 [==============================] - 0s 8ms/step - loss: 0.2532 - accuracy: 0.6193 - val_loss: 0.4565 - val_accuracy: 0.0000e+00
Epoch 6/80
48/48 [==============================] - 0s 9ms/step - loss: 0.2322 - accuracy: 0.6547 - val_loss: 0.4158 - val_accuracy: 0.0235
Epoch 7/80
48/48 [==============================] - 0s 8ms/step - loss: 0.2137 - accuracy: 0.6802 - val_loss: 0.4063 - val_accuracy: 0.0824

In [17]:
modelv2 = Sequential()
modelv2.add(Embedding(max_words, 8, input_length=maxlen))
modelv2.add(Flatten())
modelv2.add(Dense(64, activation='relu'))
modelv2.add(Dense(7, activation='sigmoid'))
modelv2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
modelv2.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 207, 8)            800000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1656)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                106048    
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 455       
Total params: 906,503
Trainable params: 906,503
Non-trainable params: 0
_________________________________________________________________


In [18]:
history2= modelv2.fit(data, y_train_one_hot,
epochs=80,
batch_size=32,
validation_split=.1)

Epoch 1/80
48/48 [==============================] - 1s 15ms/step - loss: 0.3493 - accuracy: 0.5341 - val_loss: 0.4521 - val_accuracy: 0.0000e+00
Epoch 2/80
48/48 [==============================] - 0s 9ms/step - loss: 0.2866 - accuracy: 0.5688 - val_loss: 0.4468 - val_accuracy: 0.0000e+00
Epoch 3/80
48/48 [==============================] - 1s 11ms/step - loss: 0.2475 - accuracy: 0.6468 - val_loss: 0.4733 - val_accuracy: 0.0000e+00
Epoch 4/80
48/48 [==============================] - 1s 11ms/step - loss: 0.2165 - accuracy: 0.6848 - val_loss: 0.4371 - val_accuracy: 0.0000e+00
Epoch 5/80
48/48 [==============================] - 1s 14ms/step - loss: 0.1951 - accuracy: 0.6946 - val_loss: 0.4312 - val_accuracy: 0.0059
Epoch 6/80
48/48 [==============================] - 1s 12ms/step - loss: 0.1787 - accuracy: 0.7333 - val_loss: 0.3301 - val_accuracy: 0.5647
Epoch 7/80
48/48 [==============================] - 1s 12ms/step - loss: 0.1655 - accuracy: 0.7746 - val_loss: 0.3327 - val_accuracy: 0.523

In [ ]:
train_df

In [ ]:
model.save_weights('../Model Objects/keras_outv2.h5')

In [ ]:
def getModel():
    my_dim = 8
    
    model = Sequential()
    model.add(Embedding(max_words, my_dim, input_length=maxlen))
    model.add(Flatten())
#    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    return model

In [ ]:
# Different configurations to try
epochs_list = [30, 60, 90]
batch_sizes = [16, 32, 64]
#embedding_dim = 8
# Dictionary to store the history of each configuration
history_dict = {}

for epochs in epochs_list:
    for batch_size in batch_sizes:
        # Get a fresh instance of the model for each run
        model = getModel()

        # Compile the model
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model
        history = model.fit(x_train, y_train_one_hot,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(x_val, y_val_one_hot),
                            verbose=2)  # Set verbose to 2 for less output

        # Save the history of this configuration
        history_dict[f"epochs_{epochs}_batch_{batch_size}"] = history.history

        # Optionally, you can print out the final validation accuracy for each configuration
        final_val_accuracy = history.history['val_accuracy'][-1]
        print(f"Epochs: {epochs}, Batch size: {batch_size}, Validation accuracy: {final_val_accuracy}")

# Now you can analyze the history_dict to see which configuration performed best


In [ ]:
history_dict